In [21]:
import foolbox as fb
import torch
import torchvision
import numpy as np
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
model.eval()
with open("imagenet_labels.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
from torchvision import transforms

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

Using cache found in C:\Users\tonyyyyy/.cache\torch\hub\pytorch_vision_v0.10.0


In [22]:
import os
from tensorflow.keras.preprocessing import image
import numpy as np

# read in the image files from different folders and each folder corresponds to a label
top = "adv/"
index = 0

data = []
label = []

for root, dirs, files in os.walk(top, topdown=False):
    for name in dirs:
        temp = os.path.join(root, name)
        for root1, dirs1, files1 in os.walk(temp, topdown=False):
            for j in files1:
                img_name = os.path.join(root1, j)
                img = image.load_img(img_name)
                data.append(img)
                label.append(int(name))
        print(name)

291
292
331
340
366
388
99


In [42]:
pred= []
for i in data:
  input_tensor = preprocess(i)
  input_batch = input_tensor.unsqueeze(0)
  input_batch = input_batch*2
  input_batch = torch.round(input_batch)
  input_batch = input_batch/2
  feature = model(input_batch)

  probabilities = torch.nn.functional.softmax(feature[0], dim=0)
  top5_prob, top5_catid = torch.topk(probabilities, 5)
  pred.append(top5_catid[0])

In [43]:
count = 0
for i in range(len(pred)):
    if (label[i] == pred[i].int()):
        count+=1

In [44]:
count

152

In [25]:
len(pred)

154

In [26]:
data = np.array(data)
label = np.array(label)

C:\Users\tonyyyyy\AppData\Local\Temp/ipykernel_11080/3072561362.py:1: FutureWarning: The input object of type 'JpegImageFile' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'JpegImageFile', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  data = np.array(data)
C:\Users\tonyyyyy\AppData\Local\Temp/ipykernel_11080/3072561362.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data = np.array(data)


In [27]:
import torchvision.models as models
import eagerpy as ep
from foolbox import PyTorchModel, accuracy, samples
from foolbox.attacks import LinfPGD

label = torch.tensor(label)

In [28]:
pre = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])
outputs = []

for i in data:
    tensor = pre(i)  #transform from (64, 1, 224, 224) to (64, 32, 224, 224)
    outputs.append(tensor[None, :])

result = torch.cat(outputs, dim=0)  #shape (64, 32*in_channels, 224, 224)


In [41]:
preprocessing = dict(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)
fmodel = PyTorchModel(model, bounds=(0, 1), preprocessing=preprocessing)
clean_acc = accuracy(fmodel, result, label)
print(f"clean accuracy:  {clean_acc * 100:.1f} %")
attack = LinfPGD()

clean accuracy:  98.1 %


In [30]:
result

tensor([[[[0.1137, 0.1216, 0.1216,  ..., 0.4980, 0.4863, 0.4824],
          [0.1098, 0.0980, 0.1020,  ..., 0.5059, 0.4941, 0.4824],
          [0.1569, 0.1059, 0.0863,  ..., 0.5098, 0.4941, 0.4824],
          ...,
          [0.4392, 0.4078, 0.4039,  ..., 0.3882, 0.5137, 0.6078],
          [0.4627, 0.4980, 0.4824,  ..., 0.5098, 0.6353, 0.4902],
          [0.4863, 0.4941, 0.5255,  ..., 0.6549, 0.6118, 0.5255]],

         [[0.1373, 0.1333, 0.1255,  ..., 0.5922, 0.5961, 0.5922],
          [0.1333, 0.1059, 0.1059,  ..., 0.6000, 0.6000, 0.5922],
          [0.1725, 0.1137, 0.0863,  ..., 0.6039, 0.6000, 0.5922],
          ...,
          [0.3608, 0.3294, 0.3255,  ..., 0.2941, 0.4196, 0.5137],
          [0.3804, 0.4157, 0.4039,  ..., 0.4157, 0.5412, 0.4000],
          [0.4000, 0.4078, 0.4392,  ..., 0.5647, 0.5176, 0.4353]],

         [[0.0980, 0.1020, 0.0941,  ..., 0.7686, 0.7804, 0.7804],
          [0.1020, 0.0824, 0.0824,  ..., 0.7765, 0.7882, 0.7804],
          [0.1608, 0.1059, 0.0784,  ..., 0

In [38]:
model.eval()
fmodel = PyTorchModel(model, bounds=(0, 1), preprocessing=preprocessing)
raw_advs, clipped_advs, success = attack(fmodel, result, label.long(), epsilons=0.05)

In [40]:
count = 0
for i in range(len(clipped_advs)):
    input_tensor = clipped_advs[i]
    input_batch = input_tensor.unsqueeze(0)
    input_batch = input_batch*2
    input_batch = torch.round(input_batch)
    input_batch = input_batch/2
    feature = model(input_batch)

    probabilities = torch.nn.functional.softmax(feature[0], dim=0)
    top_prob, top_catid = torch.topk(probabilities, 1)
    id = label[i]
    if (id == top_catid):
        count = count + 1
print("The fraction correctly labeled image is: ")
print(count/len(label))

The fraction correctly labeled image is: 
0.8441558441558441


In [31]:
iter = []

count = 0
for i in range(len(result)):

    input_tensor = outputs[i]
    input_batch = input_tensor*2
    input_batch = torch.round(input_batch)
    input_batch = input_batch/2
    iter.append(input_batch)


In [32]:
iter = torch.cat(iter, dim=0)

In [37]:
import torch.optim as optim
import torch.nn as nn

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.1)
for epoch in range(2):  # loop over the dataset multiple times
    running_loss = 0.0
    optimizer.zero_grad()
    outputs = model(iter)
    loss = criterion(outputs, label.long())
    loss.backward()
    optimizer.step()
    print(loss.item())

0.08323557674884796
0.07452831417322159
